In [50]:
from collections import Counter
from math import log
from random import shuffle
from statistics import mean
import pandas as pd
from names import util
from names import ngram
from tqdm import tqdm

Still confused about $P$ vs $J$

$N$ = number of characters for set of names

$n$ = number of times a character appears in a set of names

$K$ = number of characters/alphabet

$s$ = length of name

$J$ = number of names in set of names

$j$ = specific  name

$w$ = specific character

$n_{j}$ = the number of occurences of character in name $j$

$p(w|j)$ = probability of character in name which is estimated as ratio $n_{j}/s$  is normalised as $\sum_{w=1}^K p(w|j) = 1$

$p(j) = 1/J$ probability that a given character $w$ appears in name $j$

Probability of occurence of character is $\sum_{j=1}^J p(w|j)p(j) = p(w)$

$p(w) = n/N$


After observing an instance of character $w$ the prob that it comes from name $j$ is given by $p(j|w)$ which can be computed as $p(j|w) = \frac{p(w|j)p(j)}{p(w)}$


Blah blah blah mutual information


 (4) $\Delta I(s) = \sum_{w=1}^{K}  p(w)  \left[ \langle \hat{H}(J|w) \rangle - H(J|w) \right] $
 
 equation 4 represent an information measure quantifying the degree of specificty of characters over names of length $s$. The entropy term $H(J|w)$ can be directly computed from the word counts across the $J$ names.
 
 $H(J|w) = - \sum_{j=1}^P \frac{n_{j}}{n} \log_{2} \frac{n_{j}}{n}$



In [ ]:
def H_J_w(character, names):
    character_counts_total = Counter("".join(names))
    H_J_w_val = 0
    for name in names:
        character_counts = Counter("".join(name))
        fraction = character_counts[character]/character_counts_total[character]
        if fraction == 0:
            continue
        H_J_w_val -= fraction*log(fraction,2)
    return H_J_w_val
    
    
def H_J_w_bigram(bigram, names: pd.Series,possible_ngrams ):
    ngrams = ngram.get_ngrams(names, n=2)

    bigram_counts = Counter(ngrams)
    H_J_w_val = 0
    for name in names:
        ngrams_name = ngram.get_ngrams(pd.Series([name]), n=2)
        name_bigram_counts = Counter(ngrams_name)
        fraction = (name_bigram_counts[bigram]+1)/(bigram_counts[bigram]+possible_ngrams.count())
        H_J_w_val -= fraction*log(fraction,2)
                                               
    return H_J_w_val

In [30]:
test = ["name"]
# Counter(ngram.get_ngrams(pd.Series(test), n=2))
pd.Series(test).count()

1

In [47]:
def H_J_w_per_character(names):
    return_dict = dict()
    for character in set("".join(names)):
        return_dict[character] = H_J_w(character, names)
    return return_dict

def H_J_w_per_bigram(names:pd.Series,bigrams):
    return_dict = dict()
    bigrams = ngram.get_possible_ngrams(n=2)
    for bigram in tqdm(bigrams):
        return_dict[bigram] = H_J_w_bigram(bigram, names,possible_ngrams=bigrams)
    return return_dict
        
def H_J_w_per_length(names):
    lengths = {len(name) for name in names}
    return_dict = dict()
    for length in lengths:
        length_names = [name for name in names if len(name) == length]
        return_dict[length] = H_J_w_per_character(length_names)
        


In [4]:
df = pd.read_parquet(f'10_sample.parquet')
            

In [5]:
AE = list(df["AE"])
AR = list(df["AR"])

In [48]:
def corrected_information(names, length):
    names_length = [name.lower() for name in names if len(name) == length]
    names_shuffled = util.shuffle_names(names_length)[length]
    names_length_shuffled = [name.lower() for name in names_shuffled if len(name) == length]
    characters = set("".join(names_length))
    character_count_total = len("".join(names_length))
    character_counts = Counter("".join(names_length))
    H_J_w_names = H_J_w_per_character(names_length)
    H_J_w_names_shuffled = H_J_w_per_character(names_length_shuffled)
    print(names_shuffled[:10])
    corrected_information = 0
    for character in characters:
        corrected_information += (character_counts[character]/character_count_total)*(H_J_w_names_shuffled[character]-H_J_w_names[character])
    return corrected_information
    
def corrected_information_bigram(names:list, length):
    possible_bigrams = ngram.get_possible_ngrams(n=2)

    names_length = [name.lower() for name in names if len(name) == length]  
    names_length = pd.Series(names_length)
    bigrams = ngram.get_ngrams(names_length, n=2)
    bigrams_unique = bigrams.unique()
    bigrams_total = bigrams.shape[0]
    bigrams_counts = Counter(bigrams)
    H_J_w_names = H_J_w_per_bigram(names_length,bigrams=possible_bigrams)

    names_shuffled= util.shuffle_names(names_length)[length]
    names_length_shuffled = pd.Series([name.lower() for name in names_shuffled if len(name) == length])
    bigrams_shuffled = ngram.get_ngrams(names_length_shuffled, n=2)
    bigrams_shuffled_total = bigrams_shuffled.shape[0]
    bigrams_shuffled_counts = Counter(bigrams_shuffled)
    bigrams_unique_shuffled = bigrams_shuffled.unique()
    H_J_w_names_shuffled = H_J_w_per_bigram(names_length_shuffled,bigrams=possible_bigrams)
    
    
    
    corrected_information = 0
    for bigram in possible_bigrams:
        fraction = (bigrams_counts[bigram]+bigrams_shuffled_counts[bigram])/(2*bigrams_total)
        corrected_information += fraction*(H_J_w_names_shuffled[bigram]-H_J_w_names[bigram])
    

    
    

In [29]:
test2 = pd.Series([1,2,3])
test3 = pd.Series([3,4])
test2.append(test3, ignore_index=True)


C:\Users\johan\AppData\Local\Temp\ipykernel_21640\401187749.py:3: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test2.append(test3, ignore_index=True)


0    1
1    2
2    3
3    3
4    4
dtype: int64

In [ ]:
corrected_information_bigram([name.lower() for name in AR if len(name) == 5],5)

  5%|████▏                                                                            | 44/841 [00:36<12:39,  1.05it/s]

In [ ]:
corrected_information_bigram([name.lower() for name in AE if len(name) == 5],5)

In [9]:
AR_shuffled= util.shuffle_names([name.lower() for name in AR if len(name) == 5])[5]

In [19]:
AE_series = pd.Series([name.lower() for name in AE])
Counter(ngram.get_ngrams(AE_series, n=2))

Counter({'^k': 53,
         'ka': 38,
         'am': 107,
         'mr': 8,
         'ra': 96,
         'an': 173,
         'n$': 142,
         '^a': 149,
         'ah': 51,
         'hm': 20,
         'ma': 118,
         'ad': 66,
         'd$': 92,
         'kh': 15,
         'hl': 5,
         'lo': 13,
         'ou': 17,
         'ud': 12,
         'me': 41,
         'ed': 42,
         '^g': 20,
         'gl': 3,
         'le': 31,
         'en': 49,
         'nn': 17,
         '^d': 26,
         'da': 34,
         'na': 71,
         'a$': 151,
         '^j': 50,
         'jo': 16,
         'oa': 4,
         'nc': 6,
         'co': 8,
         'ol': 14,
         'll': 27,
         'li': 40,
         'in': 67,
         'ne': 27,
         'e$': 49,
         '^i': 17,
         'il': 41,
         'ly': 11,
         'yn': 6,
         '^s': 145,
         'sa': 85,
         'mi': 42,
         'ih': 4,
         'h$': 61,
         '^m': 112,
         'mo': 33,
         'oh': 29,
         'ha

In [31]:
mean(H_J_w_per_character([name.lower() for name in AR if len(name) == 5]).values())


4.2046310755221725

In [30]:
mean(H_J_w_per_character(AR_shuffled).values())

4.133309015512278

In [37]:
mean(H_J_w_per_character([name.lower() for name in AE if len(name) == 5]).values())

4.762387223267469

In [24]:
H_J_w_per_character([name.lower() for name in AE if len(name) == 5])

{'q': 1.584962500721156,
 'z': 4.175735869100492,
 'c': 3.321928094887362,
 'p': 3.875,
 'w': 3.169925001442312,
 'j': 4.321928094887363,
 'f': 3.169925001442312,
 't': 4.66449777920046,
 'k': 4.857980995127571,
 'm': 6.162890436798801,
 'r': 6.278464292480902,
 'b': 4.186704345910024,
 'o': 4.993237149412737,
 'u': 4.906890595608519,
 'h': 5.930737337562883,
 'a': 7.41726653309052,
 'l': 5.611456608683871,
 'x': 1.584962500721156,
 'd': 5.573893517584596,
 'i': 6.789784284662099,
 'v': 3.906890595608518,
 'e': 5.855485357162551,
 'y': 5.271928094887364,
 'n': 6.4599625007211605,
 's': 6.049190599108366,
 'g': 3.7004397181410926}

In [58]:
Counter([len(name) for name in AR]).most_common()

[(6, 188),
 (5, 180),
 (4, 158),
 (7, 120),
 (8, 66),
 (9, 39),
 (11, 39),
 (12, 38),
 (13, 34),
 (3, 33),
 (14, 33),
 (10, 22),
 (15, 18),
 (16, 16),
 (2, 5),
 (17, 4),
 (18, 2),
 (21, 2),
 (19, 1),
 (24, 1),
 (23, 1)]

In [59]:
Counter([len(name) for name in AE]).most_common()

[(5, 249),
 (6, 188),
 (4, 169),
 (7, 104),
 (8, 64),
 (3, 57),
 (9, 43),
 (11, 29),
 (10, 23),
 (12, 19),
 (14, 19),
 (2, 14),
 (13, 7),
 (15, 4),
 (16, 3),
 (20, 2),
 (17, 2),
 (22, 1),
 (19, 1),
 (23, 1),
 (1, 1)]

In [25]:
df

,AE,AF,AL,AO,AR,AT,AZ,BD,BE,BF
0,Kamran,Sayyed Ali Sajjad,Santiago,Jilton,Carmen,Tc,Milosh,Md,Derrick,Catherine
1,Ahmad,myr,Ali,Frederico Da Silva,Manobrava,Mario,Rufet,Engr,Jessie,Fagale
2,Khloud Ahmed,Adam,Valter,Alfredo Antonio,Cambios,Martina,Omer,Maiukh,Maurice,Syan
3,Glenn,Assadullah,Veledin,Victor,Tony,Er,Gunel,Rajdip,Sally,Bibata
4,Dana,Fahad,Marko,Samara,Martha,Anita,Orxan,Nur,Tracey,Benjamin
...,...,...,...,...,...,...,...,...,...,...
995,Shirantha,HfyZ,Edona,Dada,Thiagogerman,Duro,Abdullayeva,Hasibul,Mara,Tahirou
996,Roy,qry bw,Adan,Elias Alexande,Juan Humberto,Thorsohn,Samira,Fatematuj Zohura Prionty,Adrianus,Abdoussafy
997,Shemeer,mzdwr,Nsjdjsj,Hernany,Lydia,Marin,Amin,Chandan,Veronique,Legma
998,Sumit,Kingstar,Golbarg,Ayad,Humberto,Burak,Xudus,Nazneen,Eszter,Mamoudou
